# Différentes approches de NLP

## Données

In [1]:
import pandas as pd
df = pd.read_csv("../data/tableau.csv")

## Mot par mot, ou dictionnaires

In [2]:
pays = ["france","allemagne","suisse"]

In [4]:
df["text"] = df["Contenu_mod"].str.lower()

In [8]:
def compter_mots(t, dictionnaire=pays):
    return {i:t.count(i) for i in dictionnaire}
df["text"].apply(compter_mots)

0      {'allemagne': 1, 'france': 0, 'suisse': 14}
1       {'allemagne': 0, 'france': 1, 'suisse': 7}
2       {'allemagne': 0, 'france': 3, 'suisse': 9}
3       {'allemagne': 1, 'france': 2, 'suisse': 5}
4       {'allemagne': 0, 'france': 2, 'suisse': 6}
                          ...                     
274     {'allemagne': 0, 'france': 3, 'suisse': 1}
275     {'allemagne': 0, 'france': 1, 'suisse': 2}
276     {'allemagne': 0, 'france': 1, 'suisse': 1}
277     {'allemagne': 0, 'france': 1, 'suisse': 1}
278     {'allemagne': 0, 'france': 0, 'suisse': 1}
Name: text, Length: 279, dtype: object

## DFM

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
modele = CountVectorizer()

In [12]:
dtm = modele.fit_transform(df["text"])

In [17]:
modele.get_feature_names_out()

array(['00', '000', '000e', ..., 'œuvre', 'œuvrent', 'œuvres'],
      dtype=object)

Pas mal d'options

In [14]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [16]:
len(fr_stop)

507

In [21]:
modele = CountVectorizer(stop_words=list(fr_stop),min_df=5)
dtm = modele.fit_transform(df["text"])
dtm.shape

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


(279, 4012)

## TF-IDF

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

modele = TfidfVectorizer(stop_words=list(fr_stop))
tfidf = modele.fit_transform(df["text"].fillna("NULL"))

/opt/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


Faire des clusters

In [29]:
from sklearn.cluster import KMeans
true_k = 4
model = KMeans(n_clusters=true_k, 
               init='k-means++', 
               max_iter=200, 
               n_init=10)
model.fit(tfidf)
df["labels"] = model.labels_

Peut être pas la meilleure échelle : décomposer au paragraphe ?

## Premiers pas avec SpaCy


### Télécharger et utiliser un modèle (un trained pipepeline)

Se familiariser avec la structure doc/span/token

In [2]:
#pip install spacy

Installer le pipeline

In [3]:
#!python -m spacy download fr_core_news_lg

In [ ]:
pipeline = spacy.load("fr_core_news_lg")

In [ ]:
doc = pipeline("Nous sommes au dernier cours de Python pour les SHS")
for i in doc:
    print(i.pos_, i.lemma_)
doc.ents

Spacy pour des usages dictionnaires

In [ ]:
from spacy.matcher import Matcher

matcher = Matcher(pipeline.vocab)

# Add the pattern to the matcher
pattern = [{"LOWER": "python"}]
matcher.add("LANGUAGE", [pattern])
pattern = [{"LOWER": "r"}] 
matcher.add("LANGUAGE", [pattern])

# Process some text
doc = pipeline("Je préfère R à Python")

# Call the matcher on the doc
matches = matcher(doc)


# Iterate over the matches
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc[start:end]
    print(matched_span.text)

Possibilité de chercher des combinaisons plus complexes

In [ ]:
matcher = Matcher(pipeline.vocab)
pattern = [
    {"LEMMA": "aimer", "POS": "VERB"},
    {"POS": "NOUN"}
]
matcher.add("LANGUAGE", [pattern])
doc = pipeline("J'aime Paython plus que R")
matches = matcher(doc)
matches

Possibilité d'avoir la distance entre deux textes

In [ ]:
# Compare two documents
doc1 = pipeline("J'aime beaucoup programmer en Python")
doc2 = pipeline("J'aime beaucoup programmer en R")
print(doc1.similarity(doc2))

### Lemmatiser et garder que les noms

Utiliser le POS

In [8]:
# Partie à faire en TP

### Identifier les entités nommées

In [ ]:
text = df["text"].iloc[0]
doc = pipeline(text)
for ent in doc.ents:
    print(ent.text, ent.label_)

Afficher les entités nommées.

In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", 
                jupyter=True)

## GliNER

Un modèle plus complexe pour l'identification d'entités

In [32]:
#!pip install gliner

In [1]:
from gliner import GLiNER

model = GLiNER.from_pretrained("urchade/gliner_multi")

# Sample text for entity prediction
text = """
La suisse est un magnifique pays. La France aussi.
"""

# Labels for entity prediction
labels = ["Pays"] # for v2.1 use capital case for better performance

# Perform entity prediction
entities = model.predict_entities(text, labels, threshold=0.5)

# Display predicted entities and their labels
for entity in entities:
    print(entity["text"], "=>", entity["label"])

/opt/anaconda3/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

suisse => Pays
France => Pays


## Analyse de sentiment

Un modèle entraîné sur la prédiction

https://huggingface.co/cmarkea/distilcamembert-base-sentiment

In [2]:
from transformers import pipeline

analyzer = pipeline(
    task='text-classification',
    model="cmarkea/distilcamembert-base-sentiment",
    tokenizer="cmarkea/distilcamembert-base-sentiment"
)

model.safetensors:   0%|          | 0.00/272M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

In [3]:
analyzer("Vous êtes impressionnants d'avoir tenu ce cours jusqu'à maintenant")

[{'label': '5 stars', 'score': 0.6956597566604614}]